In [36]:
!pip install -U azureml-sdk

     |████████████████████████████████| 24.0MB 11kB/s  eta 0:00:01    |█▎                              | 993kB 3.6MB/s eta 0:00:07     |██▊                             | 2.0MB 3.6MB/s eta 0:00:07     |█████████████▉                  | 10.3MB 1.7MB/s eta 0:00:09�███████████▍               | 12.3MB 2.1MB/s eta 0:00:06     |███████████████████             | 14.3MB 2.0MB/s eta 0:00:05     |███████████████████▎            | 14.5MB 2.0MB/s eta 0:00:05
     |████████████████████████████████| 1.0MB 3.8MB/s eta 0:00:01
     |████████████████████████████████| 204kB 8.6MB/s eta 0:00:01
     |████████████████████████████████| 71kB 2.0MB/s eta 0:00:011


ERROR: azureml-widgets 1.0.48 has requirement azureml-core==1.0.48.*, but you'll have azureml-core 1.0.55 which is incompatible.
ERROR: azureml-widgets 1.0.48 has requirement azureml-telemetry==1.0.48.*, but you'll have azureml-telemetry 1.0.55 which is incompatible.
ERROR: azureml-train-automl 1.0.48 has requirement azureml-core==1.0.48.*, but you'll have azureml-core 1.0.55 which is incompatible.
ERROR: azureml-train-automl 1.0.48 has requirement azureml-pipeline-core==1.0.48.*, but you'll have azureml-pipeline-core 1.0.55 which is incompatible.
ERROR: azureml-train-automl 1.0.48 has requirement azureml-telemetry==1.0.48.*, but you'll have azureml-telemetry 1.0.55 which is incompatible.
ERROR: azureml-contrib-opendatasets 1.0.45 has requirement azureml-telemetry==1.0.45.*, but you'll have azureml-telemetry 1.0.55 which is incompatible.
ERROR: azureml-contrib-notebook 1.0.48 has requirement azureml-core==1.0.48.*, but you'll have azureml-core 1.0.55 which is incompatible.
  Found exis

In [1]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-telemetry 1.0.45 (/home/nbuser/anaconda3_501/lib/python3.6/site-packages), Requirement.parse('azureml-telemetry==1.0.48.*')).


SDK version: 1.0.48


In [2]:
from azureml.core import Workspace
ws = Workspace(subscription_id="ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9", resource_group="DatasetTestRG", workspace_name="Dataset-Test-WS")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FMEZTU8ZY to authenticate.
Interactive authentication successfully completed.
Dataset-Test-WS
DatasetTestRG
eastus
ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9


In [3]:
from azureml.core.compute import AmlCompute
compute_target_name = "nlp-test"
compute_target = AmlCompute(ws, compute_target_name)

In [4]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core.environment import DockerSection
runconfig = RunConfiguration(framework="python")
runconfig.environment.python.conda_dependencies = CondaDependencies('dependencies.yml')
runconfig.environment.docker = DockerSection()
runconfig.environment.docker.enabled = True
runconfig.environment.docker.gpu_support = True
runconfig.environment.docker.base_image = azureml.core.runconfig.DEFAULT_GPU_IMAGE

In [5]:
from azureml.data.data_reference import DataReference
from azureml.core import Datastore
ds = Datastore.get(ws, 'workspaceblobstore')

In [6]:
# data module
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

train_input_data = DataReference(
    datastore=ds,
    data_reference_name="train_input_data",
    path_on_datastore="train/cleaned_train_sampled.csv")
# train_input_data = DataReference(
#     datastore=ds,
#     data_reference_name="train_input_data",
#     path_on_datastore="debug/train_16.csv")
train_data = PipelineData("train_data",datastore=ds)

load_train_data_step = PythonScriptStep(
    name="load_data_step",
    script_name="run_modules/data_module.py", 
    arguments=["--input_data", train_input_data, 
               "--output_dir", train_data],
    inputs=[train_input_data],
    outputs=[train_data],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)

In [7]:
# data module —— load test dataset
test_input_data = DataReference(
    datastore=ds,
    data_reference_name="test_input_data",
    path_on_datastore="test/cleaned_test_data_labels.csv")
# test_input_data = DataReference(
#     datastore=ds,
#     data_reference_name="test_input_data",
#     path_on_datastore="debug/test_32.csv")

test_data = PipelineData("test_data",datastore=ds)

load_test_data_step = PythonScriptStep(
    name="load_test_data_step",
    script_name="run_modules/data_module.py", 
    arguments=["--input_data", test_input_data, 
               "--output_dir", test_data],
    inputs=[test_input_data],
    outputs=[test_data],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)

In [8]:
#split data module
# dataset1 = PipelineData("output_data1",datastore=ds)
# dataset2 = PipelineData("output_data2",datastore=ds)
# #parameters
# row_split_rate = 0.7
# random = True
# random_seed = 200
# split_data_step = PythonScriptStep(
#     name="split_data_step",
#     script_name="run_modules/split_data_module.py", 
#     arguments=["--input_dir", train_data, 
#                "--output_dir1", dataset1,
#                "--output_dir2", dataset2,
#                "--row_split_rate", row_split_rate,
#                "--random", random,
#                "--random_seed", random_seed],
#     inputs=[train_data],
#     outputs=[dataset1, dataset2],
#     compute_target=compute_target,
#     runconfig = runconfig,
#     source_directory='.'
# )


In [9]:
dataset1 = train_data
dataset2 = test_data

In [10]:
# bert module
chosed_model = PipelineData("chosed_bert",datastore=ds)

language = "En"
uncased = True
build_bert_step = PythonScriptStep(
    name="build_bert_step",
    script_name="run_modules/bert_module.py", 
    arguments=["--out_model_dir", chosed_model,
               "--language", language,
              "--uncased", uncased],
    outputs=[chosed_model],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)


In [11]:
# custom added layer module
label_num = 6
#category = 'lstm_multi_label_classifier' 
layer_name = 'multi_label_classifier'
added_layer = PipelineData("added_layer",datastore=ds)
custom_layer_step = PythonScriptStep(
    name="custom_layer_step",
    script_name="run_modules/add_layer_module.py", 
    arguments=["--label_num", label_num, 
               "--layer_name", layer_name,
               "--output_dir", added_layer],
    outputs=[added_layer],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)


In [12]:
# train module 
bert_dir = DataReference(
    datastore=ds,
    data_reference_name="bert_dir",
    path_on_datastore="models")

trained_model_dir = PipelineData("trained_model_dir", datastore=ds)
is_training_bert= False # Device cannot support training and is generally setted False.
train_batch_size = 2
use_gpu = True
num_gpu_cores = 4
learning_rate = 5e-5
num_train_epochs = 3.0
train_column_names ="comment_text"
label_column_names = "toxic severe_toxic obscene threat insult identity_hate"
train_step = PythonScriptStep(
    name="train_step",
    script_name="run_modules/train_module.py", 
    arguments=["--bert_dir", bert_dir,
               "--bert_model", chosed_model, 
               "--added_layer_config", added_layer,
               "--train_data", dataset1,
               "--output_dir", trained_model_dir,
               "--is_training_bert", is_training_bert,
               "--train_batch_size", train_batch_size,
               "--use_gpu",use_gpu,
               "--num_gpu_cores", num_gpu_cores,
               "--learning_rate", learning_rate,
               "--num_train_epochs", num_train_epochs,
               "--train_column_names",train_column_names,
               "--label_column_names", label_column_names],
    inputs=[bert_dir, chosed_model, added_layer, dataset1],
    outputs=[trained_model_dir],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)



In [13]:
# score/predict module

predict_result_dir = PipelineData("predict_dir",datastore=ds)
predict_batch_size = 2
use_gpu = True
num_gpu_cores = 4
predict_column_names = "comment_text"
predict_step = PythonScriptStep(
    name="predict_step",
    script_name="run_modules/predict_module.py", 
    arguments=["--bert_dir", bert_dir,
               "--bert_model", chosed_model, 
               "--added_layer_config", added_layer,
               "--test_data", dataset2,
               "--trained_model_dir", trained_model_dir,
               "--output_dir", predict_result_dir,
               "--predict_batch_size", predict_batch_size,
               "--use_gpu", use_gpu,
               "--num_gpu_cores", num_gpu_cores,
               "--predict_column_names", predict_column_names],
    inputs=[bert_dir, chosed_model, added_layer, dataset2, trained_model_dir],
    outputs=[predict_result_dir],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)

In [14]:
# evaluate module
evaluate_result_dir = PipelineData("evaluate_results",datastore=ds)
pr = True
roc = True
label_columns = "toxic severe_toxic obscene threat insult identity_hate"
probability_columns = "probability_1 probability_2 probability_3 probability_4 probability_5 probability_6"
evaluate_step = PythonScriptStep(
    name="evaluate_step",
    script_name="run_modules/evaluate_module.py", 
    arguments=[ "--input_data", predict_result_dir,
                "--pr", pr,
                "--roc", roc,
                "--output_dir", evaluate_result_dir,
                "--label_columns", label_columns,
                "--probability_columns", probability_columns],
    inputs=[predict_result_dir],
    outputs=[evaluate_result_dir],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)

In [15]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
pipeline_toxic= Pipeline(workspace=ws, steps=[evaluate_step])
#pipeline_split_data = Pipeline(workspace=ws, steps=[split_data_step])


In [16]:
pipeline_run = Experiment(ws, 'toxic_comment').submit(pipeline_toxic)
#pipeline_run = Experiment(ws, 'toxic_comment').submit(pipeline_split_data)

Created step evaluate_step [590fe763][b9176d67-4aed-410b-80d6-89c3f933dd76], (This step will run and generate new outputs)
Created step predict_step [d592246d][f15cd6bc-a06a-40c6-b5ae-549436b9cf2d], (This step will run and generate new outputs)
Created step build_bert_step [db396121][886e48b3-0b5b-4814-ab9c-ec95d672b9ac], (This step will run and generate new outputs)
Created step custom_layer_step [aa2febf4][8cc00054-0f03-49a9-bfbe-cac706c99976], (This step will run and generate new outputs)
Created step load_test_data_step [4e6d4541][7d4f42ca-3b8c-4a36-a807-58770e391984], (This step will run and generate new outputs)
Created step train_step [f4893916][745cd1a0-8ac4-48d7-879d-1d5f1d1196d6], (This step will run and generate new outputs)
Created step load_data_step [78e0e9b6][cd934aad-5138-48e1-b587-b910c87cdf74], (This step will run and generate new outputs)
Using data reference bert_dir for StepId [83f28508][a34cc5da-bebe-4d82-90ea-cdb21b91cc5f], (Consumers of this data are eligible to

In [35]:
from azureml.core import Experiment
from azureml.pipeline.core import PipelineRun
pipeline_run = Experiment(ws, 'toxic_comment')
pipeline_run = PipelineRun(pipeline_run, "44b1f188-beb3-4cd6-9300-23667970394a")
pipeline_run.cancel()
